In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN-LSTM4006"

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../../../src')
sys.path.append('../../../src/models')

In [3]:
import models.CNN_lstm_model as CNN_lstm_model
import training  
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

## Model

In [5]:
## Model Parameter
n_variables= 12
learning_rate = 0.001
batch_size = 10

pooling_stride = [4, 2, 2, 2, 1, 1]
weights_shape = [[7,n_variables, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32]]

bias_shape = [32, 32, 32, 32, 32, 32, 32]
activation = tf.nn.relu

lstm_n_hidden = 32
n_classes = 1


#positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [6]:
pos_weight

3.186018221237171

In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 


In [10]:
model = CNN_lstm_model.Model(n_variables, n_classes)

In [11]:
model.build_cnn_lstm_net(pooling_stride,
                         weights_shape,
                         bias_shape,
                         activation,
                         lstm_n_hidden,
                         n_classes,
                         learning_rate,
                         pos_weight)

In [12]:
train_obj = training.Trainer(model,
                 train_data, train_gt,
                 validation_data, validation_gt,
                 sess,
                 summary_path,
                 batch_size,
                 global_step = 0)

In [13]:
train_obj.train(max_iter = 10000,
                train_eval_step=100,
                validation_eval_step=100,
                display_step=100)

Tarin Iter 0, auc= 0.168709, fmeasure_max= 0.360518, precision= 0.220158, recall= 0.994643
Train Random guess:  , auc= 0.228514fmeasure_max= 0.360319, precision= 0.228875, recall= 0.846429
Validation Iter 0, auc= 0.220764, fmeasure_max= 0.387585, precision= 0.252332, recall= 0.835334
Validation Random guess:  auc= 0.229424, fmeasure_max= 0.371101, precision= 0.227823, recall= 1.000000
Minibatch Loss= 1.669174
Tarin Iter 100, auc= 0.546547, fmeasure_max= 0.555459, precision= 0.528239, recall= 0.585635
Train Random guess:  , auc= 0.198119fmeasure_max= 0.350549, precision= 0.212524, recall= 1.000000
Validation Iter 100, auc= 0.486349, fmeasure_max= 0.501241, precision= 0.392614, recall= 0.692967
Validation Random guess:  auc= 0.229347, fmeasure_max= 0.374597, precision= 0.230647, recall= 0.996569
Minibatch Loss= 0.854565
Tarin Iter 200, auc= 0.656874, fmeasure_max= 0.625418, precision= 0.592710, recall= 0.661947
Train Random guess:  , auc= 0.231004fmeasure_max= 0.361600, precision= 0.2207

In [ ]:
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=train_obj.global_step)

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path+'-1000')

In [16]:
probability_of_pos = nn_eval.evaluate_model(model,
                                   sess,
                                   test_data,
                                   test_gt)

Testing auc = 0.924815 F1_max = 0.853407 prec = 0.875466 rec = 0.832432
Test Random guess: auc= 0.227757 fmeasure_max= 0.375635 , precision= 0.231250 , recall= 1.000000


In [18]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [20]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})